In [1]:
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [2]:
@tool
def multiply(a:int , b:int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [3]:
print(multiply.invoke({'a':3 , 'b':6}))

18


In [4]:
multiply.name

'multiply'

In [5]:
multiply.description

'Given 2 numbers a and b this tool returns their product'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

Tool Binding

In [7]:
llm = ChatOpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [ ]:
llm_with_tools = llm.bind_tools([multiply])

AttributeError: 'OllamaLLM' object has no attribute 'bind_tools'

In [8]:
llm_with_tools.invoke("Hi how are you")

NameError: name 'llm_with_tools' is not defined

In [9]:
llm_with_tools.invoke("can you multiply 3 with 10")

NameError: name 'llm_with_tools' is not defined

In [13]:
query = HumanMessage("can you multiply 2 with 10")

In [14]:
message = [query]

In [15]:
message

[HumanMessage(content='can you multiply 2 with 10', additional_kwargs={}, response_metadata={})]

In [10]:
result.tool_calls[0]

NameError: name 'result' is not defined

In [11]:
multiply.invoke(result.tool_calls[0]['args'])

NameError: name 'result' is not defined

# Currency Conversion Tool

In [29]:
# Tool Create
from langchain_core.tools import InjectedToolArg
from typing import Annotated


@tool
def get_conversion_factor(base_currency: str , target_currency:str) -> float:
    """ 
    This function fetches the currency conversion factor between a given base currency and target currency 
    """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

    response = requests.get(url=url)
    return response.json

@tool
def convert(base_currency_value: int , conversion_rate: Annotated[float , InjectedToolArg]) -> float:
    """
    given a currency conversion rate this function calculates the target currency value from a given base currency value 
    """

    return base_currency_value * conversion_rate


In [30]:
get_conversion_factor.invoke({'base_currency':'USD' , 'target_currency':'INR'})

<bound method Response.json of <Response [200]>>

In [31]:
convert.invoke({'base_currency_value':10 , 'conversion_rate':85.16})

851.5999999999999

In [32]:
# tool binding
llm = ChatOpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [33]:
llm_with_tools = llm.bind_tools([get_conversion_factor , convert])

NameError: name 'llm' is not defined

In [34]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr')]

In [35]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [36]:
ai_message = llm_with_tools.invoke(messages)

NameError: name 'llm_with_tools' is not defined

In [37]:
ai_message.tool_calls

NameError: name 'ai_message' is not defined

In [38]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)


NameError: name 'ai_message' is not defined